In [1]:
# Aim - 
# 1. Fetch a user's podcast data 
# 2. Based on a search term, get all podcasts in the Spotify catalog

# Code Reference - https://github.com/sam-brady/spotify-podcasts/blob/master/Spotify%20Podcasts.ipynb

# Spotify Docs - https://developer.spotify.com/documentation/web-api/reference/episodes/
#                https://developer.spotify.com/community/news/2020/03/20/introducing-podcasts-api/

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import urllib.parse

import spotipy.util as util

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

In [2]:
def authenticate_token(scope):
    client_id = '<ENTER CLIENT ID>'         
    client_secret = '<ENTER CLIENT SECRET>'       
    username = '<ENTER USERNAME>'                                
    redirect_uri = '<ENTER REDIRECT URI>'

    return util.prompt_for_user_token(username=username, 
                                       scope=scope, 
                                       client_id=client_id,   
                                       client_secret=client_secret,     
                                       redirect_uri=redirect_uri)


In [ ]:
def get_my_podcasts(endpoint_url, token):
    '''
    Get all the podcasts a user has subscribed to
    '''
    name_list = []                                         
    id_list = []
    desc_list = []
    pub_list = []
    lang_list = []

    type = 'show'    
    market  = 'US'
    limit = 50                                             
    offset = 0

    more_runs = 1                                          
    counter = 0 

    while((offset <= 1950) & (counter <= more_runs)):
        query = f'{endpoint_url}'
        query += f'&type={type}'
        query += f'&offset={offset}'                       
        query += f'&market={market}'
        query += f'&limit={limit}'

        response = requests.get(query,                                           
                       headers={"Content-Type":"application/json", 
                                "Authorization":f"Bearer {token}"})  
        json_response = response.json()                                           

        for i in range(len(json_response['items'])):
            name_list.append(json_response['items'][i]['show']['name']) 
            id_list.append(json_response['items'][i]['show']['id']) 
            desc_list.append(json_response['items'][i]['show']['description']) 
            pub_list.append(json_response['items'][i]['show']['publisher']) 
            lang_list.append(json_response['items'][i]['show']['languages']) 

        more_runs = (json_response['total'] // 50 )                 
        counter += 1                                                    
        offset = offset + 50              

    return pd.DataFrame({'id':id_list, 'name':name_list, 'publisher':pub_list, 
                         'description':desc_list, 'languages':lang_list})

In [ ]:
token = authenticate_token('user-library-read')
my_podcasts = get_my_podcasts("https://api.spotify.com/v1/me/shows?", token)

In [3]:
def search_podcasts(search, endpoint_url, token):
    '''
    Based on a search term, get all the podcasts in the Spotify catalog
    '''
    name_list = []                                         
    id_list = []
    desc_list = []
    pub_list = []
    lang_list = []

    type = 'show'    
    market  = 'US'
    limit = 50                                             
    offset = 0                                             

    more_runs = 1                                          
    counter = 0  
                                                    
    while((offset <= 1950) & (counter <= more_runs)):           
        query = f'{endpoint_url}'
        query += f'&q={search}'
        query += f'&type={type}'
        query += f'&offset={offset}'                       
        query += f'&market={market}'
        query += f'&limit={limit}'

        response = requests.get(query,                                          
                       headers={"Content-Type":"application/json", 
                                "Authorization":f"Bearer {token}"})  
        json_response = response.json()                                           

        for i in range(len(json_response['shows']['items'])):                      
            id_list.append(json_response['shows']['items'][i]['id'])               
            name_list.append(json_response['shows']['items'][i]['name'])           
            desc_list.append(json_response['shows']['items'][i]['description'])
            pub_list.append(json_response['shows']['items'][i]['publisher'])
            lang_list.append(json_response['shows']['items'][i]['languages'])

        more_runs = (json_response['shows']['total'] // 50 )                
        counter += 1                                                    
        offset = offset + 50                                           

    return pd.DataFrame({'id':id_list, 'name':name_list, 'publisher':pub_list,
                         'description':desc_list,'languages':lang_list})

In [4]:
token = authenticate_token('playlist-modify-public playlist-modify')
ds_podcasts = search_podcasts('data science', "https://api.spotify.com/v1/search?", token)
finance_podcasts = search_podcasts('finance', "https://api.spotify.com/v1/search?", token)
politics_podcasts = search_podcasts('politics', "https://api.spotify.com/v1/search?", token)